**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.optimizers import adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=0}^{T-1}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

Learning the optimal policy $\pi^*$ induces a trade-off between exploiting the current policy $\pi$ and exploring the environment by taking actions that are not necessarily in line with the current policy. Here ``epsilon`` is the probability of taking a random action. In other words it fosters exploration. If the randomly generated sample is below ``epsilon``, we explore the environment by taking a random action, else we use the action according to the current policy.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0: # up
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1: # down
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2: # right
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3: # left
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=30 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The arrays ``position`` and ``board`` are used to form the ``state``. Indeed, the ``state`` is a $5$ by $5$ grid on two channels. The first channel is the ``board`` that is visible for the agent, while the second channel is the ``position`` that is visible for the agent.

The ``board`` array contains the *cheese* and *poisonous* cells. The ``bonus`` represents the *cheese* cells while the ``malus`` represents the *poisonous* cells. The ``bonus`` and ``malus`` masks are then mapped onto the ``board`` array.

The ``position`` array contains the allowed positions on the board. ``-1`` cells represent *walls*, ``0`` cells are allowed, while the ``1`` cell represents the agent position.

**Note:** I corrected a small mistake in the ``position`` variable assignment done in ``Environment.reset`` as can be seen above.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        """ Act randomly from a given state s
        it proposes an random action a"""
        # Randomly select an action
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix='', epsilon_greedy=True):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        state = env.reset()
        game_over = False
        win = 0
        lose = 0
        
        while not game_over:
            if epsilon_greedy: # Are we using sticky actions (i.e. random action with probability ϵ)?
                action = agent.act(state, train=True)
            else: # Else, no sticky action. This is helpful to enhance the expert DQN
                action = agent.act(state, train=False)
            prev_state = state
            state, reward, game_over = env.act(action)
            if reward > 0:
                win += reward
            if reward < 0:
                lose -= reward
        
        # Save as a mp4
        env.draw(prefix+str(e + 1))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random1.mp4'))

########################################################
### Note: ``mp4`` videos cannot be played on Firefox ###
########################################################

Win/lose count 10.0/20.0. Average score (-10.0)
Win/lose count 13.5/15.0. Average score (-5.75)
Win/lose count 7.0/26.0. Average score (-10.166666666666666)
Win/lose count 7.5/20.0. Average score (-10.75)
Win/lose count 12.5/14.0. Average score (-8.9)
Win/lose count 5.5/10.0. Average score (-8.166666666666666)
Win/lose count 10.0/13.0. Average score (-7.428571428571429)
Win/lose count 10.0/17.0. Average score (-7.375)
Win/lose count 12.5/20.0. Average score (-7.388888888888889)
Win/lose count 5.0/11.0. Average score (-7.25)
Win/lose count 12.5/12.0. Average score (-6.545454545454546)
Win/lose count 12.0/13.0. Average score (-6.083333333333333)
Win/lose count 15.0/20.0. Average score (-6.0)
Win/lose count 7.0/12.0. Average score (-5.928571428571429)
Win/lose count 8.5/12.0. Average score (-5.766666666666667)
Win/lose count 11.5/17.0. Average score (-5.75)
Win/lose count 10.0/12.0. Average score (-5.529411764705882)
Win/lose count 8.0/17.0. Average score (-5.722222222222222)
Win/lose cou

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**Answer:**


1.
First, let us rewrite $Q^{\pi}(s, a)$ now that $T = \infty$:

$$Q^{\pi}(s,a) = E_{p^{\pi}}[\sum_{t=0}^{\infty} \gamma^t r(s_t, a_t) | s_0 = s, a_0 = a]$$

We can now write:

\begin{align}
Q^{\pi}(s,a) & =  E_{p^{\pi}}[\gamma^0 r(s_0, a_0) + \sum_{t=1}^{\infty} \gamma^t r(s_t, a_t) | s_0 = s, a_0 = a] \\
  & = r(s, a) + \gamma \sum_{s', a'} p(s', a' | s, a) E_{p^{\pi}}[\sum_{t=1}^{\infty} \gamma^{t-1} r(s_t, a_t) | s_1 = s', a_1 = a'] \\
  & = r(s, a) + \gamma E_{(s', a') \sim p(. |s,a)}[Q^{\pi}(s', a')] \\
\end{align}

Hence:

$$\boxed{Q^{\pi}(s,a) = E_{(s', a') \sim p(. |s,a)}[r(s, a) + \gamma Q^{\pi}(s', a')]}$$


2.
By definition of $Q^*(s,a) = \max_{\pi}Q^{\pi}(s,a)$, we get:

\begin{align}
Q^*(s,a) & =  \max_{\pi}Q^{\pi}(s,a) \\
  & = \max_{\pi} E_{(s', a') \sim p(. |s,a)}[r(s, a) + \gamma Q^{\pi}(s', a')] \\
  & =  \max_{\pi} \sum_{(s', a')} [r(s,a) + \gamma Q^{\pi}(s', a')] p(s',a'|s,a)\\
  & = \sum_{s'} [r(s, a) + \gamma \max_{a'} Q^*(s', a')] \pi^*(s' |s, a) \\
\end{align}

Hence we obtain Bellman's optimality equation:

$$\boxed{Q^*(s,a) = E_{s' \sim \pi^*(.|s,a)}[r(s,a) + \gamma \max_{a'}Q^*(s',a')]}$$

3.
At this point we can turn this equation into an update. Until the optimal policy $\pi^*$ is reached, there will always be a difference between $y_t = r(s, a) + \gamma \max_{a'} Q(s', a', \theta)$ and $Q(s, a, \theta)$ where $s' \sim \pi^t(. | s, a)$ and $\pi^t$ is the current policy. Therefore we can derive a plausible objective:

$$\boxed{\mathcal{L}(\theta) = E_{s' \sim \pi(. | s, a)} \big[||r(s, a) + \gamma \max_{a'} Q(s', a', \theta) - Q(s, a, \theta)||^2\big]}$$

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        memory_size = len(self.memory)
        if memory_size == self.max_memory:
            self.memory.pop(0) # Remove first element
        self.memory.append(m)
        

    def random_access(self):
        # Randomly select an index in the memory buffer
        ix = np.random.randint(0, len(self.memory))
        # Select the move
        m = self.memory[ix]
        return m

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if (e + 1) % 10 == 0:
            env.draw(prefix+str(e + 1))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e + 1, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        # Get the model output
        out = self.model.predict(s.reshape(1, 5, 5, self.n_state))
        # Return the action that maximizes the output
        return np.argmax(out)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            # Randomly access the memory
            state, next_state, action, reward, game_over = self.memory.random_access()
            input_states[i] = state
            # Get the current model prediction on the current state 
            target_q[i] = self.model.predict(state.reshape(1, 5, 5, self.n_state))
            # Given the action taken, update the target with the next state prediction when the game is on
            if game_over:
                ######## FILL IN
                target_q[i, action] = reward
            else:
                ######## FILL IN
                target_q[i, action] = reward + self.discount * np.max(self.model.predict(next_state.reshape(1, 5, 5, self.n_state)))
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        # Train the model on the moves batch
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        #model.compile("sgd", "mse")
        model.compile("adam", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state,)))
        model.add(Dense(30, activation="linear")) # linear activations work better than ReLU
        model.add(Dense(10, activation="linear"))
        model.add(Dense(4))
        
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile("adam", "mse") # Use default ADAM instead
        self.model = model
        print(self.model.summary())
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, epsilon = 0.2, memory_size=2000, batch_size = 32) # default learning rate lr=0.001
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train30.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 30)                1530      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                310       
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 44        
Total params: 1,884
Trainable params: 1,884
Non-trainable params: 0
_________________________________________________________________
None
Epoch 001/030 | Loss 0.0289 | Win/lose count 7.5/5.0 (2.5)
Epoch 002/030 | Loss 0.0317 | Win/lose count 9.5/4.0 (5.5)
Epoch 003/030 | Loss 0.0103 | Win/lose count 4.5/6.0 (-1.5)
Epoch 004/030 | Loss 0.0060 | Win/lose count 6.0/5.0 (1.0)
Epoch 005/030 | Loss 0.0026 | 

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        # Note: plain linear activation were found to work better in practice
        model.add(Conv2D(16, kernel_size=3, input_shape=(5, 5, self.n_state,), activation="linear"))
        model.add(Conv2D(8, kernel_size=3, activation="linear"))
        model.add(Flatten())
        model.add(Dense(4))
        
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile("adam", "mse")
        self.model = model
        print(self.model.summary())

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, epsilon = 0.2, memory_size=2000, batch_size = 32) # default learning rate lr=0.001
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train30.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 3, 3, 16)          304       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 8)           1160      
_________________________________________________________________
flatten_2 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 36        
Total params: 1,500
Trainable params: 1,500
Non-trainable params: 0
_________________________________________________________________
None
Epoch 001/030 | Loss 0.0098 | Win/lose count 3.0/11.0 (-8.0)
Epoch 002/030 | Loss 0.0120 | Win/lose count 6.0/2.0 (4.0)
Epoch 003/030 | Loss 0.0065 | Win/lose count 5.5/7.0 (-1.5)
Epoch 004/030 | Loss 0.0053 | Win/lose count 10.5/12.0 (-1.5)
Epoch 005/030 | Loss 0.00

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=.45)
agent_cnn = DQN_CNN(size, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, epsilon = 0.1, memory_size=2000, batch_size = 32)
#agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json') # typo here
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 3, 16)          304       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 1, 1, 8)           1160      
_________________________________________________________________
flatten_3 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 36        
Total params: 1,500
Trainable params: 1,500
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 50)                0         
_________________________________________________________________
den

In [16]:
HTML(display_videos('cnn_test20.mp4'))

In [17]:
HTML(display_videos('fc_test20.mp4'))

**Comment:**

* The ``temperature`` corresponds to the likelihood of a cell to contain either a malus or a bonus. When it decreases, there are fewer malus or bonus cells. In other words, the rewards are sparse. Conversely, when it is equal to 1, then all cells are bonus cells (since bonus cells suppress malus cells by design of the environment). In the testing experiment above, we increased by 0.15 (from 0.3 to 0.45).

* What we observe is the tendency of the agent to get stuck in certain loops. However since we currently use the ``epsilon``-greedy exploration even in the testing phase, we get out of these loops from time to time. That being said, we still observe the agent exploring only a small part of the environment.

* In addition, the vision range of the agent is fairly limited. This means that given a sparse reward environment and a poorly learned policy, the agent may never see any bonus or malus cells and be stuck in an edge of the grid. We even notice that behavior for the fully connected DQN on the maximal temperature once it has cleared a large reward zone, it does not know where to go next.

* Regardless of the ``temperature`` variable, we notice that the CNN DQN performs **similarly** to the fully-connected one while having fewer parameters. Notice that we corrected a bug where ``agent_fc`` was not loading its trained weights.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [18]:
def train_explore(agent,env,epoch,prefix='', epsilon_decay=0.9, env_explore=False):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        # Add the epsilon decay (similar to a learning rate decay) to reduce
        # exploration as training goes
        agent.set_epsilon(agent.epsilon * (epsilon_decay ** e))

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            if env_explore: # Are we using EnvironmentExploring?
                state, reward, game_over = env.act(action, train=True)
            else: # If not, back to standard environment to compare
                state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if (e + 1) % 10 == 0:
            env.draw(prefix+str(e + 1))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              " | Agent epsilon: {:.6f}"
              .format(e + 1, epoch, loss, win, lose, win-lose, agent.epsilon))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [19]:
# Training
#env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
env = Environment(grid_size=size, max_time=T, temperature=0.3) # We only implement the ϵ-greedy exploration first
#agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
agent = DQN_CNN(size, epsilon = 0.5, memory_size=2000, batch_size = 32,n_state=2)
train_explore(agent, env, epochs_train, prefix='cnn_train_eps_explore', epsilon_decay=0.99)
HTML(display_videos('cnn_train_eps_explore30.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 3, 3, 16)          304       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 1, 1, 8)           1160      
_________________________________________________________________
flatten_5 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_9 (Dense)              (None, 4)                 36        
Total params: 1,500
Trainable params: 1,500
Non-trainable params: 0
_________________________________________________________________
None
Epoch 001/030 | Loss 0.0732 | Win/lose count 3.0/9.0 (-6.0) | Agent epsilon: 0.500000
Epoch 002/030 | Loss 0.0042 | Win/lose count 3.5/4.0 (-0.5) | Agent epsilon: 0.495000
Epoch 003/030 | Loss 0.0019 | Win/lose count 6.0/3.0 (3.0) | Agent epsilon: 0.485149
Epoch 004/

In [20]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_eps_explore', epsilon_greedy=True) # ϵ-greedy exploration during testing
HTML(display_videos('cnn_test_eps_explore20.mp4'))

Win/lose count 2.5/0. Average score (2.5)
Win/lose count 2.5/0. Average score (2.5)
Win/lose count 6.0/0. Average score (3.6666666666666665)
Win/lose count 2.5/0. Average score (3.375)
Win/lose count 7.5/0. Average score (4.2)
Win/lose count 3.5/0. Average score (4.083333333333333)
Win/lose count 6.0/0. Average score (4.357142857142857)
Win/lose count 4.0/0. Average score (4.3125)
Win/lose count 4.5/0. Average score (4.333333333333333)
Win/lose count 1.5/1.0. Average score (3.95)
Win/lose count 3.5/2.0. Average score (3.727272727272727)
Win/lose count 6.5/1.0. Average score (3.875)
Win/lose count 3.5/0. Average score (3.8461538461538463)
Win/lose count 4.5/0. Average score (3.892857142857143)
Win/lose count 1.0/0. Average score (3.7)
Win/lose count 3.5/1.0. Average score (3.625)
Win/lose count 5.5/0. Average score (3.735294117647059)
Win/lose count 1.5/1.0. Average score (3.5555555555555554)
Win/lose count 0.5/0. Average score (3.3947368421052633)
Win/lose count 4.5/0. Average score (3

In [21]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_eps_explore', epsilon_greedy=False) # No ϵ-greedy exploration during test
HTML(display_videos('cnn_test_eps_explore20.mp4'))

Win/lose count 2.5/0. Average score (2.5)
Win/lose count 3.0/0. Average score (2.75)
Win/lose count 1.5/0. Average score (2.3333333333333335)
Win/lose count 0/0. Average score (1.75)
Win/lose count 5.5/0. Average score (2.5)
Win/lose count 3.0/1.0. Average score (2.4166666666666665)
Win/lose count 1.5/0. Average score (2.2857142857142856)
Win/lose count 0/0. Average score (2.0)
Win/lose count 0.5/0. Average score (1.8333333333333333)
Win/lose count 7.0/0. Average score (2.35)
Win/lose count 4.5/0. Average score (2.5454545454545454)
Win/lose count 4.0/0. Average score (2.6666666666666665)
Win/lose count 1.0/0. Average score (2.5384615384615383)
Win/lose count 2.5/0. Average score (2.5357142857142856)
Win/lose count 2.5/0. Average score (2.533333333333333)
Win/lose count 1.0/0. Average score (2.4375)
Win/lose count 5.0/0. Average score (2.588235294117647)
Win/lose count 2.0/0. Average score (2.5555555555555554)
Win/lose count 2.0/0. Average score (2.526315789473684)
Win/lose count 1.0/0.

**Comment:**

* Using the decreasing $\epsilon$-greedy scheme alone, we see that the agent is able to explore a significantly larger area of the map and it performs almost flawlessly after 30 epochs during training. Besides, it almost always avoids *poisonous* cells during the testing phase when we do not use $\epsilon$-greedy scheme in the testing phase. However the testing average score results remain below the results we obtained not using a decreasing $\epsilon$-greedy policy (although we were using a higher ``temperature`` in our previous tests). In addition, using sticky actions in training seems to help the agent overcome certain loops and get a slightly better average final score than when removing the sticky actions entirely.

* Now we add the EnvironmentExploring that indicates whether a state has been visited.

In [22]:
class EnvironmentExploring(Environment):
    def __init__(self, *args, **kwargs):
        super(EnvironmentExploring, self).__init__(*args, **kwargs)
    # Environment.draw and Environment.get_frame do not change
    
    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0: # up
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1: # down
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2: # right
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3: # left
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        
        #reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    
    
    #if False:
    ## use those samples of code:
    #In train explore:
    #state, reward, game_over = env.act(action, train=True)

    ## In Environment exploring:
    # You will have to change n_state to 3 because you will use one more layer!
    #reward = 0
    #if train:
    #    reward = -self.malus_position[self.x, self.y]
    #self.malus_position[self.x, self.y] = 0.1

    #reward = reward + self.board[self.x, self.y]
    # 3 "feature" states instead of 2
    #state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
    #                                self.board.reshape(self.grid_size, self.grid_size,1),
    #                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [23]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
#env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, epsilon = 0.5, memory_size=2000, batch_size = 32,n_state=3)
#agent = DQN_CNN(size, lr=.1, epsilon = 0.5, memory_size=2000, batch_size = 32,n_state=2)
train_explore(agent, env, epochs_train, prefix='cnn_train_full_explore', epsilon_decay=0.99) 
HTML(display_videos('cnn_train_full_explore30.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 3, 3, 16)          448       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 1, 1, 8)           1160      
_________________________________________________________________
flatten_6 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 36        
Total params: 1,644
Trainable params: 1,644
Non-trainable params: 0
_________________________________________________________________
None
Epoch 001/030 | Loss 0.0131 | Win/lose count 1.5/6.0 (-4.5) | Agent epsilon: 0.500000
Epoch 002/030 | Loss 0.0095 | Win/lose count 14.5/12.0 (2.5) | Agent epsilon: 0.495000
Epoch 003/030 | Loss 0.0097 | Win/lose count 15.0/14.0 (1.0) | Agent epsilon: 0.485149
Epoch 0

In [24]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_full_explore_no_eps_test', epsilon_greedy=True) # ϵ-greedy exploration during testing
HTML(display_videos('cnn_test_full_explore_no_eps_test20.mp4'))

Win/lose count 2.0/1.0. Average score (1.0)
Win/lose count 2.0/0. Average score (1.5)
Win/lose count 2.0/0. Average score (1.6666666666666667)
Win/lose count 3.0/0. Average score (2.0)
Win/lose count 3.0/0. Average score (2.2)
Win/lose count 1.0/0. Average score (2.0)
Win/lose count 3.0/1.0. Average score (2.0)
Win/lose count 2.0/0. Average score (2.0)
Win/lose count 7.5/0. Average score (2.611111111111111)
Win/lose count 6.5/0. Average score (3.0)
Win/lose count 1.5/0. Average score (2.8636363636363638)
Win/lose count 7.5/0. Average score (3.25)
Win/lose count 1.0/0. Average score (3.076923076923077)
Win/lose count 5.0/0. Average score (3.2142857142857144)
Win/lose count 5.0/0. Average score (3.3333333333333335)
Win/lose count 10.5/2.0. Average score (3.65625)
Win/lose count 2.0/1.0. Average score (3.5)
Win/lose count 15.0/0. Average score (4.138888888888889)
Win/lose count 11.5/0. Average score (4.526315789473684)
Win/lose count 1.5/0. Average score (4.375)
Final score: 4.375


**Comment:**

By adding in the count-based exploration scheme, we obtain similar results on the training phase. Even when using the $\epsilon$-greedy scheme in the testing phase, we obtain slightly better results on average when using the count-based exploration AND the $\epsilon$-greedy exploration approaches together.

Our last experiment consists in using only the count-based exploration but no $\epsilon$-greedy approach in both training and testing phases.

In [25]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
#env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0, memory_size=2000, batch_size = 32,n_state=3) # no ϵ-greedy exploration during training
#agent = DQN_CNN(size, lr=.1, epsilon = 0.5, memory_size=2000, batch_size = 32,n_state=2)
train_explore(agent, env, epochs_train, prefix='cnn_train_count_explore', epsilon_decay=0.99)
HTML(display_videos('cnn_train_count_explore30.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 3, 3, 16)          448       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 1, 1, 8)           1160      
_________________________________________________________________
flatten_7 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 4)                 36        
Total params: 1,644
Trainable params: 1,644
Non-trainable params: 0
_________________________________________________________________
None
Epoch 001/030 | Loss 0.0001 | Win/lose count 4.0/4.0 (0.0) | Agent epsilon: 0.000000
Epoch 002/030 | Loss 0.0017 | Win/lose count 2.5/1.0 (1.5) | Agent epsilon: 0.000000
Epoch 003/030 | Loss 0.0038 | Win/lose count 0.5/0 (0.5) | Agent epsilon: 0.000000
Epoch 004/030 

In [26]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_count_explore', epsilon_greedy=True) # ϵ-greedy exploration during testing
HTML(display_videos('cnn_test_count_explore20.mp4'))

Win/lose count 1.5/0. Average score (1.5)
Win/lose count 3.5/1.0. Average score (2.0)
Win/lose count 0.5/1.0. Average score (1.1666666666666667)
Win/lose count 0.5/1.0. Average score (0.75)
Win/lose count 0.5/0. Average score (0.7)
Win/lose count 0.5/0. Average score (0.6666666666666666)
Win/lose count 3.0/0. Average score (1.0)
Win/lose count 1.0/0. Average score (1.0)
Win/lose count 1.5/2.0. Average score (0.8333333333333334)
Win/lose count 1.5/0. Average score (0.9)
Win/lose count 3.0/0. Average score (1.0909090909090908)
Win/lose count 3.5/1.0. Average score (1.2083333333333333)
Win/lose count 1.0/0. Average score (1.1923076923076923)
Win/lose count 2.0/1.0. Average score (1.1785714285714286)
Win/lose count 2.5/1.0. Average score (1.2)
Win/lose count 3.5/0. Average score (1.34375)
Win/lose count 1.5/0. Average score (1.3529411764705883)
Win/lose count 1.5/0. Average score (1.3611111111111112)
Win/lose count 2.0/1.0. Average score (1.3421052631578947)
Win/lose count 0/0. Average sco

**Comment:**

* Here we observe that the count-based exploration approach on its own performs worse than the $\epsilon$-greedy approach on its own and worse than both approaches simultaneously. The count-based exploration is not sufficient on its own.

* That being said, note that the number of test environments is fairly small (here 20), so the results should be taken with caution as there is significant variance. One should in fact conduct multiple runs and plot the standard deviations. In addition we are only observing the results using the same ``temperature`` in both training and testing phases.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

**Comment:**

* In this section we explore training a model that imitates an expert DQN. This can be seen as an instance of Imitation Learning where agents learn from domain (often human) experts.

* Moreover, it may relate to the fact that shallow networks might be able to learn complex function by using deeper original models as mentors (see [paper](https://arxiv.org/abs/1312.6184) from Lei Jimmy Ba and Rich Caruana where the authors conclude: "Our success in training shallow neural nets to mimic deeper models suggests that there probably exist better algorithms for training shallow feed-forward nets than those currently available.")

* In addition, using the output of a fixed and randomly initialized network has recently being used in Reinforcement Learning to tackle sparse reward environments. The authors (Yuri Burda, Harrison Edwards, Amos Storkey and Oleg Klimov) present an intrinsic reward (as opposed to extrinsinc reward given by the environment) that acts as a curiosity module and that is essentially the prediction error on the output of a fixed and randomly intialized network. They coin this technique as [Random Network Distillation](https://arxiv.org/pdf/1810.12894.pdf) and achieve state-of-the-art on the infamous Montezuma's Revenge game from the Atari Learning Environment that is considered as an exploration benchmark since the rewards are very rare (i.e. sparse).

In [27]:
class DQN_imitate(DQN):
    def __init__(self, mentor, *args, lr=0.1,**kwargs):
        super(DQN_imitate, self).__init__( *args,**kwargs)
        
        # NN Model
        self.mentor = mentor
        
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state,)))
        model.add(Dense(30, activation="linear"))
        model.add(Dense(10, activation="linear"))
        model.add(Dense(4))
        
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile("adam", "mse") # default ADAM
        self.model = model
        print(self.model.summary())
        
    # Overwrite reinforce
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        # Use the agent own memory
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            state, next_state, action, reward, game_over = self.memory.random_access()
            input_states[i] = state
            #target_q[i] = self.model.predict(state.reshape(1, 5, 5, self.n_state))
            # Use the mentor model predictions
            target_q[i] = self.mentor.model.predict(state.reshape(1, 5, 5, self.n_state))
            
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l 

In [28]:
# Mentor training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
#env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, epsilon = 0.5, memory_size=2000, batch_size = 32,n_state=3)
#agent = DQN_CNN(size, lr=.1, epsilon = 0.5, memory_size=2000, batch_size = 32,n_state=2)
train_explore(agent, env, epochs_train, prefix='cnn_train_full_explore', epsilon_decay=0.99) 
HTML(display_videos('cnn_train_full_explore30.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 3, 3, 16)          448       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 1, 1, 8)           1160      
_________________________________________________________________
flatten_8 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_12 (Dense)             (None, 4)                 36        
Total params: 1,644
Trainable params: 1,644
Non-trainable params: 0
_________________________________________________________________
None
Epoch 001/030 | Loss 0.0119 | Win/lose count 5.0/4.0 (1.0) | Agent epsilon: 0.500000
Epoch 002/030 | Loss 0.0093 | Win/lose count 2.5/5.0 (-2.5) | Agent epsilon: 0.495000
Epoch 003/030 | Loss 0.0071 | Win/lose count 6.0/4.0 (2.0) | Agent epsilon: 0.485149
Epoch 004/0

In [29]:
# Learner training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
learner = DQN_imitate(mentor=agent, grid_size=size, epsilon=0.5, memory_size=2000, batch_size=32, n_state=3) # mentor = expert DQN CNN agent
train_explore(learner, env, epochs_train, prefix="fc_train_imitate", epsilon_decay=0.99)
HTML(display_videos("fc_train_imitate30.mp4"))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          (None, 75)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 30)                2280      
_________________________________________________________________
dense_14 (Dense)             (None, 10)                310       
_________________________________________________________________
dense_15 (Dense)             (None, 4)                 44        
Total params: 2,634
Trainable params: 2,634
Non-trainable params: 0
_________________________________________________________________
None
Epoch 001/030 | Loss 0.1686 | Win/lose count 16.5/15.0 (1.5) | Agent epsilon: 0.500000
Epoch 002/030 | Loss 0.0810 | Win/lose count 10.0/8.0 (2.0) | Agent epsilon: 0.495000
Epoch 003/030 | Loss 0.0269 | Win/lose count 12.5/9.0 (3.5) | Agent epsilon: 0.485149
Epoch 00

In [30]:
# Evaluation
test(learner,env,epochs_test,prefix='fc_test_imitate', epsilon_greedy=True) # ϵ-greedy exploration during testing
HTML(display_videos('fc_test_imitate20.mp4'))

Win/lose count 3.5/1.0. Average score (2.5)
Win/lose count 3.5/0. Average score (3.0)
Win/lose count 2.0/0. Average score (2.6666666666666665)
Win/lose count 6.0/1.0. Average score (3.25)
Win/lose count 2.5/0. Average score (3.1)
Win/lose count 1.0/0. Average score (2.75)
Win/lose count 9.0/0. Average score (3.642857142857143)
Win/lose count 3.5/0. Average score (3.625)
Win/lose count 2.0/0. Average score (3.4444444444444446)
Win/lose count 0/0. Average score (3.1)
Win/lose count 3.0/0. Average score (3.090909090909091)
Win/lose count 0.5/0. Average score (2.875)
Win/lose count 13.0/0. Average score (3.6538461538461537)
Win/lose count 14.5/1.0. Average score (4.357142857142857)
Win/lose count 5.0/0. Average score (4.4)
Win/lose count 2.0/0. Average score (4.25)
Win/lose count 2.5/0. Average score (4.147058823529412)
Win/lose count 0.5/0. Average score (3.9444444444444446)
Win/lose count 1.5/0. Average score (3.8157894736842106)
Win/lose count 11.0/0. Average score (4.175)
Final score: 

**Comment:**

* The learner tends to get stuck and not explore much (this is striking if we omit the $\epsilon$-greedy exploration in testing.

* It is pointless to compare these results to the FC DQN network trained from scratch as the latter was then evaluated on a different ``temperature`` (0.45 vs 0.3). 

* Note that our implementation directly exploits the expert Q function and not only the action taken at a given state, hence it does not imitate the mentor in the same way as it would if it were only given the action performed by the mentor given the state. Here we have access to the internal model of the expert, which is not the case in practice.

* In the following section we learn by using strictly the mentor memory instead of trying to imitate it by having access to its internal model.

***

In [31]:
class DQN_imitate(DQN):
    def __init__(self, mentor, *args, lr=0.1,**kwargs):
        super(DQN_imitate, self).__init__( *args,**kwargs)
        
        # NN Model
        self.mentor = mentor
        
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state,)))
        model.add(Dense(30, activation="linear"))
        model.add(Dense(10, activation="linear"))
        model.add(Dense(4))
        
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile("adam", "mse")
        self.model = model
        print(self.model.summary())
    
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        # We save the state in the memory but do not use it
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            # Use mentor memory
            state, next_state, action, reward, game_over = self.mentor.memory.random_access()
            input_states[i] = state
            # Get the current model prediction on the current state 
            target_q[i] = self.model.predict(state.reshape(1, 5, 5, self.n_state))
            # Given the action taken, update the target with the next state prediction when the game is on
            if game_over:
                ######## FILL IN
                target_q[i, action] = reward
            else:
                ######## FILL IN
                target_q[i, action] = reward + self.discount * np.max(self.model.predict(next_state.reshape(1, 5, 5, self.n_state)))
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

In [32]:
# Learner training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
learner = DQN_imitate(mentor=agent, grid_size=size, epsilon=0.5, memory_size=2000, batch_size=32, n_state=3) # mentor = expert DQN CNN agent
train_explore(learner, env, epochs_train, prefix="fc_train_imitate", epsilon_decay=0.99)
HTML(display_videos("fc_train_imitate30.mp4"))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_10 (Flatten)         (None, 75)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 30)                2280      
_________________________________________________________________
dense_17 (Dense)             (None, 10)                310       
_________________________________________________________________
dense_18 (Dense)             (None, 4)                 44        
Total params: 2,634
Trainable params: 2,634
Non-trainable params: 0
_________________________________________________________________
None
Epoch 001/030 | Loss 0.0616 | Win/lose count 7.5/16.0 (-8.5) | Agent epsilon: 0.500000
Epoch 002/030 | Loss 0.0107 | Win/lose count 12.0/10.0 (2.0) | Agent epsilon: 0.495000
Epoch 003/030 | Loss 0.0042 | Win/lose count 8.0/9.0 (-1.0) | Agent epsilon: 0.485149
Epoch 0

In [33]:
# Evaluation
test(learner,env,epochs_test,prefix='fc_test_imitate', epsilon_greedy=True) # ϵ-greedy exploration during testing
HTML(display_videos('fc_test_imitate20.mp4'))

Win/lose count 3.0/0. Average score (3.0)
Win/lose count 5.0/0. Average score (4.0)
Win/lose count 6.0/0. Average score (4.666666666666667)
Win/lose count 0.5/0. Average score (3.625)
Win/lose count 5.5/0. Average score (4.0)
Win/lose count 3.0/0. Average score (3.8333333333333335)
Win/lose count 0.5/0. Average score (3.357142857142857)
Win/lose count 4.0/0. Average score (3.4375)
Win/lose count 6.0/0. Average score (3.7222222222222223)
Win/lose count 2.5/0. Average score (3.6)
Win/lose count 5.5/0. Average score (3.772727272727273)
Win/lose count 2.5/0. Average score (3.6666666666666665)
Win/lose count 6.0/0. Average score (3.8461538461538463)
Win/lose count 0.5/0. Average score (3.607142857142857)
Win/lose count 7.5/0. Average score (3.8666666666666667)
Win/lose count 3.5/0. Average score (3.84375)
Win/lose count 1.0/0. Average score (3.676470588235294)
Win/lose count 2.5/0. Average score (3.611111111111111)
Win/lose count 1.5/0. Average score (3.5)
Win/lose count 0.5/0. Average scor

**Comment:**

This time we only use the mentor's memory to obtain its actions/moves and train the learner model to imitate its behavior. As expected, since we do not rely on the expert internal model anymore, it is harder to imitate it by only using its actions/moves, thus we obtain a slightly worse average score.